<a href="https://colab.research.google.com/github/nvrancovich/gh22-twitter-PLN/blob/main/notebooks/menciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import date
pd.set_option('display.max_colwidth', None)

In [ ]:
url = 'https://drive.google.com/file/d/1IklQ716OMB-ZYETa4NZM6Eq7cJnrDzOK/view?usp=share_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url, index_col=0)

In [ ]:
df.head(1)

,Date,Tweet
0,2022-12-08 11:03:54+00:00,Porque siento que Agustín puede irse con el mismo o más porcentaje que Juan? #GH2022 #GH22


La hora está adelantada 3 horas respecto a la de Argentina

In [ ]:
df.Date = pd.to_datetime(df.Date)
df.Date[0]

Timestamp('2022-12-08 11:03:54+0000', tz='UTC')

In [ ]:
df.Date = df.Date - pd.Timedelta(hours=3)
df.Date[0]

Timestamp('2022-12-08 08:03:54+0000', tz='UTC')

Cambiando el Date a solo fecha


In [ ]:
df.Date = df.Date.dt.date

In [ ]:
df.groupby('Date').count()

,Tweet
Date,
2022-12-04,22961
2022-12-05,26451
2022-12-06,16631
2022-12-07,29432
2022-12-08,4525


In [ ]:
24000*60

1440000

Sacando las palabras más frequentes

In [ ]:
import numpy as np

In [ ]:
def filtro(tweet):
  contador = 0
  mencionados = []

  participantes = {
    'marcos':['marcos','marquitos','primo'],
    'tomas':['tomás','holder','tomas'],
    'julieta':['julieta','juli'],
    'coti':['coti','coty'],
    'daniela':['daniela','dani','pestañela'],
    'nacho':['nacho'],
    'agustin':['agustin','agustín','frodo','agus'],
    'maxi':['maxi'],
    'alexis':['alexis','cone','conejo'],
    'walter':['walter','alfa'],
    'romina':['romina','romi'],
    'thiago':['thiago'],
    'maria':['maria','cata'],
    'mora':['mora'],
    'lucila':['lucila','tora'],
    'juan':['juan'],
    'juliana':['juliana','tini']
  }

  for participante in participantes:
    for apodo in participantes[participante]:
      if apodo in tweet:
        contador += 1
        mencionados.append(participante)
        break
  if len(mencionados) == 0:
    return np.NaN
  else:
    return mencionados

In [ ]:
df['Tweet'] = df['Tweet'].str.lower()
df['mencionados'] = df['Tweet'].apply(filtro)

In [ ]:
df = df.dropna()

In [ ]:
filas = df[['Date','mencionados']].to_dict('records')
filas[:5]

[{'Date': datetime.date(2022, 12, 8), 'mencionados': ['agustin', 'juan']},
 {'Date': datetime.date(2022, 12, 8), 'mencionados': ['romina']},
 {'Date': datetime.date(2022, 12, 8),
  'mencionados': ['julieta', 'coti', 'agustin']},
 {'Date': datetime.date(2022, 12, 8),
  'mencionados': ['marcos', 'agustin', 'thiago']},
 {'Date': datetime.date(2022, 12, 8), 'mencionados': ['maxi', 'alexis']}]

In [ ]:
conteo = []

for fila in filas:
  for mencionado in fila['mencionados']:
    conteo.append({'fecha':fila['Date'], 'mencionado':mencionado, 'menciones':1})

In [ ]:
df = pd.DataFrame(conteo).groupby(['fecha','mencionado']).count().reset_index()
df.head()

,fecha,mencionado,menciones
0,2022-12-04,agustin,3087
1,2022-12-04,alexis,268
2,2022-12-04,coti,738
3,2022-12-04,daniela,5496
4,2022-12-04,juan,72


In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
df

,date,mencionado,conteo
0,2022-12-04,agustin,3087
1,2022-12-04,alexis,268
2,2022-12-04,coti,738
3,2022-12-04,daniela,5496
4,2022-12-04,juan,72
...,...,...,...
80,2022-12-08,nacho,405
81,2022-12-08,romina,347
82,2022-12-08,thiago,561
83,2022-12-08,tomas,21


In [ ]:
fig = px.line(df, x="fecha", y="menciones", color='mencionado')
fig.show()